In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def conv_block(input_tensor, num_filters):
    """Blocos convolucionais com Conv2D + BatchNormalization + Activation"""
    x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def encoder_block(input_tensor, num_filters):
    """Bloco do codificador com conv_block + MaxPooling"""
    x = conv_block(input_tensor, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(input_tensor, concat_tensor, num_filters):
    """Bloco do decodificador com Conv2DTranspose + Concatenation + conv_block"""
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)
    x = layers.concatenate([x, concat_tensor], axis=-1)
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """Constrói a rede U-Net"""
    inputs = layers.Input(shape=input_shape)

    # Encoder
    x1, p1 = encoder_block(inputs, 16)
    x2, p2 = encoder_block(p1, 32)
    x3, p3 = encoder_block(p2, 64)
    x4, p4 = encoder_block(p3, 128)

    # Ponte
    bridge = conv_block(p4, 256)

    # Decoder
    d1 = decoder_block(bridge, x4, 128)
    d2 = decoder_block(d1, x3, 64)
    d3 = decoder_block(d2, x2, 32)
    d4 = decoder_block(d3, x1, 16)

    # Output
    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(d4)

    model = tf.keras.Model(inputs, outputs)
    return model

# Construir e compilar o modelo
input_shape = (256, 256, 3)  # Supondo imagens RGB de 256x256
model = build_unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


2024-05-23 16:52:15.165150: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-05-23 16:52:15.165183: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-23 16:52:15.165190: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-23 16:52:15.165483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-23 16:52:15.165502: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
import numpy as np

# Simular dados de entrada (imagens)
x_train = np.random.random((10, 256, 256, 3))
y_train = np.random.randint(0, 2, (10, 256, 256, 1))

# Simular dados de validação
x_val = np.random.random((2, 256, 256, 3))
y_val = np.random.randint(0, 2, (2, 256, 256, 1))


In [3]:
# Treinar o modelo
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10


2024-05-23 16:52:27.852276: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.5002 - loss: 0.8158 - val_accuracy: 0.5023 - val_loss: 0.6932
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step - accuracy: 0.5119 - loss: 0.7833 - val_accuracy: 0.5020 - val_loss: 0.6932
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.5220 - loss: 0.7602 - val_accuracy: 0.5027 - val_loss: 0.6932
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.5315 - loss: 0.7423 - val_accuracy: 0.5030 - val_loss: 0.6932
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.5401 - loss: 0.7278 - val_accuracy: 0.5031 - val_loss: 0.6932
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.5487 - loss: 0.7153 - val_accuracy: 0.5024 - val_loss: 0.6932
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.5571 - loss: 0.7041 - val_accuracy: 0.5015 - val_loss: 0.6932
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.5657 - loss: 0.6940 - val_accuracy: 0.5020 - val_loss: 0.6932
Epoc

In [5]:
import cv2
import numpy as np
import json
from shapely.geometry import Polygon

def mask_to_polygons(mask, threshold=0.5):
    """Converte uma máscara binária em uma lista de polígonos (formato WKT)"""
    # Binarizar a máscara
    mask = (mask >= threshold).astype(np.uint8)
    # Encontrar contornos
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for contour in contours:
        # Simplificar contornos para reduzir o número de pontos
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) > 2:  # Um polígono válido tem pelo menos 3 pontos
            poly = Polygon(approx.squeeze())
            polygons.append(poly.wkt)
    return polygons

def create_json(polygons, feature_type="building"):
    """Cria um objeto JSON a partir de uma lista de polígonos WKT"""
    features = []
    for i, polygon in enumerate(polygons):
        feature = {
            "properties": {
                "feature_type": feature_type,
                "uid": str(i)  # ID único simples, considerar um gerador de UUID real
            },
            "wkt": polygon
        }
        features.append(feature)
    return json.dumps({"features": features}, indent=4)

# Simulando a saída do modelo (máscara binária)
output_mask = np.random.rand(256, 256) > 0.7  # Exemplo de saída binária

# Converter máscara para polígonos
polygons = mask_to_polygons(output_mask)

# Criar JSON
output_json = create_json(polygons)

print(output_json)


{
    "features": [
        {
            "properties": {
                "feature_type": "building",
                "uid": "0"
            },
            "wkt": "POLYGON ((224 254, 223 255, 224 255, 224 254))"
        },
        {
            "properties": {
                "feature_type": "building",
                "uid": "1"
            },
            "wkt": "POLYGON ((187 254, 187 255, 189 255, 188 254, 187 254))"
        },
        {
            "properties": {
                "feature_type": "building",
                "uid": "2"
            },
            "wkt": "POLYGON ((149 255, 148 254, 147 254, 146 255, 145 255, 146 255, 147 254, 148 255, 149 255))"
        },
        {
            "properties": {
                "feature_type": "building",
                "uid": "3"
            },
            "wkt": "POLYGON ((129 254, 129 255, 130 255, 129 254))"
        },
        {
            "properties": {
                "feature_type": "building",
                "uid": "4"
     